# E-Mission to Scorpio

In [1]:
import emission.core.get_database as edb
import pandas as pd
import requests
import copy
import json
import enum
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.timeseries.timequery as estt
from transport_co2 import estimate_co2
from datetime import datetime
import uuid
import requests
import copy
import json
from uuid import UUID
import arrow


Connecting to database URL localhost


In [2]:
class PredictedModeTypes(enum.IntEnum):
    UNKNOWN = 0
    WALKING = 1
    BICYCLING = 2
    BUS = 3
    TRAIN = 4
    CAR = 5
    AIR_OR_HSR = 6
    SUBWAY = 7
    TRAM = 8
    LIGHT_RAIL = 9
    
mode_mapping = {
    PredictedModeTypes.UNKNOWN: "walk",
    PredictedModeTypes.WALKING: "walk",
    PredictedModeTypes.BICYCLING: "walk",
    PredictedModeTypes.BUS: "bus",
    PredictedModeTypes.TRAIN: "light_rail",
    PredictedModeTypes.CAR: "small_car",
    PredictedModeTypes.AIR_OR_HSR: "airplane",
    PredictedModeTypes.SUBWAY: "subway",
    PredictedModeTypes.TRAM: "light_rail",
    PredictedModeTypes.LIGHT_RAIL: "light_rail"
}

def compute_carbon_footprint(x):
    mode = mode_mapping[x.sensed_mode]
    return estimate_co2(mode=mode, distance_in_km=x.distance/1000)


ngsi_template = {
   "id": "urn:section1",
   "type": "SectionObserved",
   "transportMode": {
       "type": "Property",
	    "observedAt": "2021-03-24T12:10:00Z",
       "value": "car"
   },	
   "distance": {
       "type": "Property",
	    "observedAt": "2021-03-24T12:10:00Z",
       "value": 3464
   },
   "duration": {
       "type": "Property",
	    "observedAt": "2021-03-24T12:10:00Z",
       "value": 123
   },
   "speed": {
       "type": "Property",
	    "observedAt": "2021-03-24T12:10:00Z",
       "value": 40
   },
   "co2": {
       "type": "Property",
	    "observedAt": "2021-03-24T12:10:00Z",
       "value": 123
   },
	"location": {
		"type": "GeoProperty",
		"value": {
			"type": "LineString",
			"coordinates": 
				[
					[139.815535, 35.772622999999996],
					[139.815535, 35.774623]
				]
		
		}
	},		
   "@context": [
		{"transportMode": "odala:transportMode", "distance": "odala:distance", "duration": "odala:duration",
         "co2": "odala:co2"},
       "http://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context.jsonld"
   ]
}

In [3]:
def create_payloads(is_df):
    payloads = []
    for index, row in is_df.iterrows():
        payload = copy.deepcopy(ngsi_template)
        observedAt = datetime.utcfromtimestamp(row["end_ts"]).strftime('%Y-%m-%dT%H:%M:%SZ')
        payload["id"] = "urn:"+ str(row["cleaned_section"])
        payload["transportMode"]["value"] = PredictedModeTypes(row["sensed_mode"]).name
        payload["transportMode"]["observedAt"] = observedAt
        payload["distance"]["value"] = row["distance"]
        payload["distance"]["observedAt"] = observedAt
        payload["duration"]["value"] = row["duration"]
        payload["duration"]["observedAt"] = observedAt
        payload["speed"]["value"] = row["speed"]
        payload["speed"]["observedAt"] = observedAt
        payload["co2"]["value"] = row["co2"]
        payload["co2"]["observedAt"] = observedAt
        payload["location"]["value"]["coordinates"] = [row["start_loc"]["coordinates"] ,row["end_loc"]["coordinates"]]
        payloads.append(payload)
    return payloads

def post_payloads(payloads):
    # url = 'http://cema.nlehd.de:2042/ngsi-ld/v1/entityOperations/upsert'
    url = 'http://cema.nlehd.de:2042/ngsi-ld/v1/entities'
    headers = {'Content-Type': 'application/ld+json'}
    for e in payloads[0:20]:
        r = requests.post(url, data=json.dumps(e), headers=headers)    
        if r.status_code != 201:
            print("request failed:", r.status_code)
            print(r.text)
    

In [4]:
all_users = list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0}))

In [5]:
new_data = False
for user in all_users:
    ts = esta.TimeSeries.get_time_series(user['uuid'])
    tq = estt.TimeQuery("data.start_ts", arrow.utcnow().float_timestamp-(3600*6), # start of range, 6 hours
                              arrow.utcnow().float_timestamp)  # end of range
    is_df = ts.get_data_df("analysis/inferred_section", time_query=tq)
    if is_df.empty:
        continue
    new_data = True
    is_df["speed"] = is_df.distance/is_df.duration
    is_df["co2"] = is_df.apply(compute_carbon_footprint, axis=1)
    payloads = create_payloads(is_df)
    post_payloads(payloads)
if not new_data:
    print("Did not find any new data")

Did not find any new data
